# Mustererkennung/Machine Learning - Assignment 8



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("iris.data", header=None)
data.head(n=5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


#### Splitting the data into training/test and according to their class memberships

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data[list(range(4))], data[4], test_size=0.2, random_state=None, stratify=data[4])

X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
X_test, y_test = X_test.to_numpy(), y_test.to_numpy()

X_train_s = X_train[y_train=='Iris-setosa']
y_train_s = np.ones((X_train_s.shape[0], ))

X_test_s = X_test[y_test=='Iris-setosa']
y_test_s = np.ones((X_test_s.shape[0], ))

X_train_vv = X_train[np.equal(y_train=='Iris-versicolor', y_train=='Iris-virginica')]
y_train_vv = np.zeros((X_train_vv.shape[0], ))

X_test_vv = X_test[np.equal(y_test=='Iris-versicolor', y_test=='Iris-virginica')]
y_test_vv = np.zeros((X_test_vv.shape[0], ))

shuffle_idx = np.random.permutation(X_train_s.shape[0])
X_train_s, y_train_s = X_train_s[shuffle_idx], y_train_s[shuffle_idx]

X_train, y_train = np.vstack((X_train_s, X_train_vv)), np.hstack((y_train_s, y_train_vv))
X_test, y_test = np.vstack((X_test_s, X_test_vv)), np.hstack((y_test_s, y_test_vv))

shuffle_idx = np.random.permutation(X_train.shape[0])
X_train, y_train = X_train[shuffle_idx], y_train[shuffle_idx]

In [4]:
def accuracy(labels, predictions):
    return np.mean(labels == predictions)

def heavyside(x):
    return np.where(x > 0, np.ones_like(x), np.zeros_like(x))

def center_data(x):
    return x - np.mean(x, axis=0)

def vector_length(x):
    return np.linalg.norm(x)

class SingleLayerPerceptron():
    def fit(self, x, y, theta, lr):
        x, y = x.copy(), y.copy()
        self.w = np.mean(x[y == 1], axis=0)
        x = center_data(x)
        while True:
            rand_idx = np.random.randint(0, x.shape[0])
            x_cur, y_cur = x[rand_idx], y[rand_idx]
            w_last = self.w
            scalar = np.dot(self.w.T, x_cur)
            if y_cur == 1 and scalar > 0:
                continue
            if y_cur == 1 and scalar <= 0:
                self.w = self.w + lr * x_cur
            if y_cur == 0 and scalar < 0:
                continue
            if y_cur == 0 and scalar >= 0:
                self.w = self.w - lr * x_cur
            if vector_length(self.w - w_last) <= theta:
                break
    
    def predict(self, x):
        x = x.copy()
        x = center_data(x)
        return heavyside(x @ self.w)
    

In [5]:
clf = SingleLayerPerceptron()
clf.fit(X_train, y_train, 10 ** -4, lr=10 ** -4)

In [6]:
y_pred = clf.predict(X_test)
print(accuracy(y_test, y_pred))

0.5


In [7]:
class DenseLayer():
    def __init__(self, input_shape, output_shape, activation):
        """
        output_shape is equal to the number of neurons in the layer
        """
        self.w = np.random.uniform(low=-1, high=1, size=(input_shape, output_shape))
        self.b = np.ones((1, output_shape))
        self.activation = activation
        
    def forward(self, x):
        x_t = np.hstack((np.ones((x.shape[0], 1)), x))
        w_t = np.vstack((self.b, self.w))
        return self.activation(x_t @ w_t)
        

In [8]:
class NeuralNetwork():
    def __init__(self, layers):
        self.layers = layers
        
    def predict(self, x):
        x = x.copy()
        for layer in self.layers:
            x = layer.forward(x)
        return x

layers = []
layers.append(DenseLayer(input_shape=X_train.shape[1], output_shape=64, activation=heavyside))
layers.append(DenseLayer(input_shape=64, output_shape=128, activation=heavyside))
layers.append(DenseLayer(input_shape=128, output_shape=64, activation=heavyside))
layers.append(DenseLayer(input_shape=64, output_shape=1, activation=heavyside))

clf = NeuralNetwork(layers)
y_pred = clf.predict(X_test)

print(y_pred)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
